In [57]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import nengo_dl
from tensorflow.python.keras import Input, Model
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.layers import Conv2D, Dropout, AveragePooling2D, Flatten, Dense, BatchNormalization, Conv3D
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, ShuffleSplit


In [58]:
# Dataset path is by default saved in dataset_result/bci_dataset.npz
dataset_path = os.path.join('dataset_result', 'bci_dataset.npz')

# Load the numpy file containing the dataset
dataset = np.load(dataset_path)
features, labels = dataset['features'], dataset['labels'] # get features and labels

print('Features shape:', features.shape)
print('Labels shape:', labels.shape)

Features shape: (2976, 14, 36, 10)
Labels shape: (2976,)


In [59]:
# Set seed for consistency
seed = 1
np.random.seed(seed)
tf.random.set_seed(seed)

In [60]:
# Convert labels to one hot encoding
cat = OneHotEncoder()
labels = labels.reshape(-1, 1)
labels = cat.fit_transform(labels).toarray()
labels = labels.reshape((labels.shape[0], 1, -1))
print(labels)
print('Labels shape:', labels.shape)

# Reshape features for the NN
features = features.reshape((features.shape[0], 1, -1))
print('Features shape:', features.shape)

[[[0. 1.]]

 [[0. 1.]]

 [[0. 1.]]

 ...

 [[0. 1.]]

 [[0. 1.]]

 [[0. 1.]]]
Labels shape: (2976, 1, 2)
Features shape: (2976, 1, 5040)


In [61]:
def cnn_model_1():
    """
    Creates a CNN neural network
    :return: tensorflow model of the ANN
    """

    inp = Input(shape=(14, 360, 1), name='input_layer')
    conv2d = Conv2D(filters=9, kernel_size=(3, 3), activation='relu')(inp)
    dropout1 = Dropout(0.5, seed=seed)(conv2d)
    avg_pooling = AveragePooling2D(pool_size=(2, 2))(dropout1)
    flatten = Flatten()(avg_pooling)
    dense1 = Dense(1000, activation='relu')(flatten)
    batch_norm = BatchNormalization()(dense1)
    dense2 = Dense(500, activation='relu')(batch_norm)
    dropout2 = Dropout(0.5, seed=seed)(dense2)
    output = Dense(2, activation='softmax', name='output_layer')(dropout2)

    return Model(inputs=inp, outputs=output)


def cnn_model_2():
    inp = Input(shape=(14, 360, 1), name='input_layer')
    conv1 = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(inp)
    dropout1 = Dropout(0.2, seed=seed)(conv1)
    avg_pool1 = AveragePooling2D(pool_size=(2, 2))(dropout1)
    conv2 = Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(avg_pool1)
    dropout2 = Dropout(0.2)(conv2)
    avg_pool2 = AveragePooling2D(pool_size=(2, 2))(dropout2)
    # conv3 = Conv2D(filters=128, kernel_size=(3, 3), activation='relu')(avg_pool2)
    # dropout3 = Dropout(0.2)(conv3)
    # avg_pool3 = AveragePooling2D(pool_size=(2, 2))(dropout3)
    flatten = Flatten()(avg_pool2)
    dense1 = Dense(512, activation='relu')(flatten)
    batch_norm = BatchNormalization()(dense1)
    dense2 = Dense(256, activation='relu')(batch_norm)
    output = Dense(2, activation='softmax', name='output_layer')(dense2)

    return Model(inputs=inp, outputs=output)


def dense_only_model():
    inp = Input(shape=(14, 360, 1), name='input_layer')
    flatten = Flatten()(inp)
    dense1 = Dense(1260, activation='relu')(flatten)
    dropout = Dropout(0.5)(dense1)
    dense2 = Dense(630, activation='relu')(dropout)
    dense3 = Dense(315, activation='relu')(dense2)
    dense4 = Dense(15, activation='relu')(dense3)
    output = Dense(2, activation='sigmoid', name='output_layer')(dense4)

    return Model(inputs=inp, outputs=output)

In [62]:


def run_ann(model, train, valid, test, params_save_path, iteration, shuffle_training=True, num_epochs=5):
    x_train, y_train = train[0], train[1]
    x_valid, y_valid = valid[0], valid[1]
    x_test, y_test = test[0], test[1]

    converter = nengo_dl.Converter(model)

    with nengo_dl.Simulator(converter.net, minibatch_size=16) as simulator:
        simulator.compile(optimizer=keras.optimizers.Adam(),
                          loss=keras.losses.BinaryCrossentropy(),
                          metrics=['accuracy'])

        input_layer = converter.inputs[model.get_layer('input_layer')]
        output_layer = converter.outputs[model.get_layer('output_layer')]

        simulator.fit(
            x={ input_layer: x_train }, y={ output_layer: y_train },
            validation_data=(
                { input_layer:x_valid }, { output_layer: y_valid }
            ),
            epochs=num_epochs,
            shuffle=shuffle_training
        )

        simulator.save_params(params_save_path)
        ann_eval = simulator.evaluate(x={ input_layer: x_test }, y={ output_layer: y_test }) # evaluate accuracy
        print('{}. ann accuracy: {:5f}%'.format(iteration, ann_eval['probe_accuracy'] * 100)) # log accuracy
        return ann_eval['probe_accuracy'] # return accuracy

In [63]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.25, random_state=seed, shuffle=True)
print('x_train shape: {}, y_train shape: {}', x_train.shape, y_train.shape)

# Create directory for ANN parameters used to construct the spiking network for Nengo
os.makedirs('nengo_params', exist_ok=True)
ann = []

num_iterations = 5 # Number of cross validation iterations
iteration = 1 # Current iteration
shuffle_split = ShuffleSplit(n_splits=num_iterations, test_size=0.25, random_state=seed) # Shuffle split for cross valid

for train, valid in shuffle_split.split(x_train):
    print('Current iteration: ', iteration)
    x_train_curr, y_train_curr = x_train[train], y_train[train]
    x_valid_curr, y_valid_curr = x_train[valid], y_train[valid]

    params_path = os.path.join('nengo_params', 'params_{}'.format(iteration))
    model = dense_only_model()

    # run ann
    ann_result = run_ann(model=model,
                         train=(x_train_curr, y_train_curr),
                         valid=(x_valid_curr, y_valid_curr),
                         test=(x_test, y_test),
                         params_save_path=params_path,
                         iteration=iteration,
                         num_epochs=30
                         )
    ann.append(ann_result)
    print(ann_result)
    iteration += 1

x_train shape: {}, y_train shape: {} (2232, 1, 5040) (2232, 1, 2)
Current iteration:  1
Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               
Epoch 1/30
104/104 [==============================] - 6s 35ms/step - loss: 0.7991 - probe_loss: 0.7991 - probe_accuracy: 0.4883 - val_loss: 0.6932 - val_probe_loss: 0.6932 - val_probe_accuracy: 0.4926

c:\dev\anaconda\envs\spiking-env\lib\site-packages\nengo_dl\converter.py:326: UserWarning: Layer type Dropout does not have a registered converter. Falling back to TensorNode.
  warnings.warn(
c:\dev\anaconda\envs\spiking-env\lib\site-packages\nengo_dl\simulator.py:460: UserWarning: No GPU support detected. See https://www.nengo.ai/nengo-dl/installation.html#installing-tensorflow for instructions on setting up TensorFlow with GPU support.
  warnings.warn(
c:\dev\anaconda\envs\spiking-env\lib\site-packages\nengo_dl\simulator.py:1921: UserWarning: Number of elements in input data (558) is not evenly divisible by Simulator.minibatch_size (16); input data will be truncated.
  warnings.warn(
c:\dev\anaconda\envs\spiking-env\lib\site-packages\nengo_dl\simulator.py:1921: UserWarning: Number of elements in input data (1674) is not evenly divisible by Simulator.minibatch_size (16); input data will be truncated.
  warnings.warn(
c:\dev\anaconda\envs\spiking-env\lib\site-packages\nengo_dl\simulat